In [1]:
# Import packages
import pandas as pd
import numpy as np
import plotly.graph_objects as go

import dash
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html

# Graph 1: Global Electrification to Achieve the 1.5C Goal

In [4]:
# Map each energy sector/carrier type to specified color
col_map = {('Transportation', 'Electricity'):'#B8DBEA',
           ('Transportation', 'On-board fuels'):'#5A8EC2',
           ('Industry', 'Electricity'):'#A3A7AD',
           ('Industry', 'On-site energy'):'#687282',
           ('Buildings/Others', 'Electricity'):'#E674B1',
           ('Buildings/Others', 'On-site energy'):'#C30C61'}

# Read Data Source
world = pd.read_csv('Data/world.csv')

# Finds sum of each energy sector demand for each year
add_total = world.groupby(['Energy Sector', 'Year']).agg({'Final Energy Demand': 'sum'})

for y in range(2015,2055, 5):
    for s in ['Transportation', 'Industry', 'Buildings/Others']:
        for t in ['Electricity', 'On-site energy']:
            if (s == 'Transportation') & (t == 'On-site energy'):
                t = 'On-board fuels'
            insert_value = add_total.loc[s, y].values # Finds the total category demand to add for year
            index = list(world.loc[(world['Energy Sector'] == s) & (world['Year'] == y) & (world['Carrier Type'] == t)].index)
            world.loc[index, 'Total'] = insert_value # Adds the total category demand to the original data frame

# Calculates the total percentage demand for a carrier in the energy category            
world['Total Percentage'] = round(world['Final Energy Demand'] / world['Total'] * 100, 2)

# Loops through the color map and adds acolor column to the dataframe 
for single in list(col_map.keys()):
    world.loc[(world['Energy Sector'] == single[0]) & (world['Carrier Type'] == single[1]), 'Color'] = col_map[single]

# Create figure
fig = go.Figure()

# Stacked area plot
for s in world['Energy Sector'].unique():
    for t in ['Electricity', 'On-site energy']:
        if (s == 'Transportation') & (t == 'On-site energy'):
            t = 'On-board fuels'
        plot_data = world.loc[(world['Energy Sector']==s) & (world['Carrier Type']==t),]
        # Adds the cumulative sum of trace to create a stacked area plot
        fig.add_trace(go.Scatter(x = plot_data['Year'],
                                 y = plot_data['Final Energy Demand'],
                                 mode = 'none',
                                 fill = 'tonexty', fillcolor = col_map[s,t],
                                 stackgroup = 'one',
                                 name = s + ' - ' + t,
                                 hoverinfo = 'none'))
        
# Add vertical lines for every five years
for y in range(2020,2050,5):
    fig.add_shape(type = 'line',
                  x0 = y, y0 = 0,
                  x1 = y, y1 = world.loc[world['Year']==y,'Final Energy Demand'].sum(),
                  line = dict(color = 'white', width = 2, dash = 'dot'))    


# Adds the hover text region, creates transparent circles at the intersection of each area
# Hover text includes information about the dataset
fig.add_trace(go.Scatter(x = world['Year'],
                         y = world['CumSumEnergy'],
                         marker = dict(color = 'white', size = 7, opacity = 0),
                         mode = 'markers',
                         name = '',
                         customdata = world[['RE Share','Final Energy Demand', 'Total Percentage', 'Total',
                                               'Energy Sector', 'Carrier Type', 'Color']],
                         hovertemplate = 'Year: <b>%{x}</b>'+
                                         '<br>Energy Carrier: <b><span style="color:%{customdata[6]}">%{customdata[4]}</b></span>'+
                                         '<br>Carrier Demand: <b>%{customdata[1]:,.0f} PJ</b>'+
                                         '<br>Energy Category: <b>%{customdata[5]}</b>'+
                                         '<br>Category Demand: <b>%{customdata[3]:,.0f} PJ</b>' + 
                                         '<br>Carrier Percentage: <b>%{customdata[2]:,.0f}%</b>'))


# Add text labels fir the Energy Sectors
for s in world['Energy Sector'].unique():
    t = 'On-site energy'
    if (s == 'Transportation') & (t == 'On-site energy'):
        t = 'On-board fuels'
    fig.add_annotation(text = f'<b>{s}</b>',
                        x = 2015.2,
                        y = world.loc[(world['Energy Sector']==s) &
                                      (world['Carrier Type']==t) &
                                      (world['Year']==2015),'CumSumEnergy'].values[0],
                        xref = 'x', xanchor = 'left', align = 'left',
                        yref = 'y', yanchor = 'top', yshift = -10,
                        showarrow = False,
                        font = dict(size = 14, color = 'white'))

# Updates the layout of the plot
fig.update_layout({'plot_bgcolor':'rgba(0, 0, 0, 0)', 'paper_bgcolor':'rgba(0, 0, 0, 0)'},
                  width = 900, height = 800,
                  margin_pad = 5,
                  font_family = 'Calibri (body)',
                  title = {'text':'<b>Global Electrification to Achieve the 1.5C Goal</b>',
                           'font_size':20,
                           'font_color':'black',
                           'y':0.9,
                           'yanchor':'top'},
                  showlegend = False,
                  xaxis = dict(range = [2015,2051],
                               tickfont = dict(color = 'black', size = 14)),
                  yaxis = dict(range = [0,370000],
                               tickmode = 'array',
                               tickvals = [50000,100000,150000,200000,250000,300000,350000],
                               ticktext = ['50,000','100,000','150,000','200,000','250,000','300,000','350,000'],
                               tickfont = dict(color = 'black', size = 14),
                               title_standoff = 25,
                               title = {'text':'<b>Final Energy Demand (PJ)</b>',
                                        'font_size':16,
                                        'font_color':'black'}),
                  hoverlabel = dict(bgcolor = 'white', font_size = 15))

# Display the figure
#fig.show()

# To pass the figure into a Dash app, comment out fig.show() above, then uncomment and run the following lines
app = JupyterDash(__name__)
app.layout = html.Div([
    dcc.Graph(figure = fig)
])

if __name__ == '__main__':
    app.run_server(mode = 'inline', debug = False, port = 8051)

127.0.0.1 - - [05/Jan/2021 16:51:25] "GET /_shutdown_dd0ac356-d233-45a4-af2f-95b28be40fd7 HTTP/1.1" 200 -
 * Running on http://127.0.0.1:8051/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Jan/2021 16:51:25] "GET /_alive_dd0ac356-d233-45a4-af2f-95b28be40fd7 HTTP/1.1" 200 -


127.0.0.1 - - [05/Jan/2021 16:51:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2021 16:51:25] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2021 16:51:25] "GET /_dash-layout HTTP/1.1" 200 -
